
# Classification of text documents using sparse features

This is an example showing how scikit-learn can be used to classify documents
by topics using a bag-of-words approach. This example uses a scipy.sparse
matrix to store the features and demonstrates various classifiers that can
efficiently handle sparse matrices.

In [4]:
# Author: Peter Prettenhofer <peter.prettenhofer@gmail.com>
#		  Olivier Grisel <olivier.grisel@ensta.org>
#		  Mathieu Blondel <mathieu@mblondel.org>
#		  Lars Buitinck
# Updated by: Kisti
# Finalized by: Kamil Veli TORAMAN <kvtoraman@kaist.ac.kr>
# 23 August 2017
# License: BSD 3 clause

from __future__ import print_function

import logging
import numpy as np
import nltk.stem
from optparse import OptionParser
import sys, copy
from time import time

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.linear_model import RidgeClassifier
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.svm import SVC

from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import Perceptron
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestCentroid
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils.extmath import density
from sklearn import metrics
from sklearn.model_selection import train_test_split

In [9]:

def transform_label(clf, prob, topk):
    rst_arr = np.empty( (len(prob), topk), dtype=object) 
    for i in range(len(prob)):
        s_items = np.argsort(prob[i])[-topk:]

        for j in range(len(s_items)):
            rst_arr[i][j] = clf.classes_[s_items[j]]
    return rst_arr

def apk_per_class(actual, predicted, k=1):
    """
    Computes the average precision at k.
    This function computes the average prescision at k between two lists of
    items per each class
    Parameters
    ----------
    actual : list
             A list of elements that are to be predicted (order doesn't matter)
    predicted : multi list
                A list of predicted elements (order does matter)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The average precision at k over the input lists
    """
    class_score = {}
    micro_correct = 0.0
    length = 0
    global cumulative_class_score
    if len(actual) == len(predicted):

        for i in range(len(actual)):
            if actual[i] not in class_score:
                class_score[actual[i]] = [0.0, 0.0] # correct_score, length
            if actual[i] not in cumulative_class_score:
                cumulative_class_score[actual[i]] = [0.0, 0.0] # correct_score, length
             
            well_classified = False
            for pred in predicted[i]:
                if actual[i] == pred:
                    class_score[actual[i]][0] += 1.0
                    micro_correct += 1.0
                    well_classified = True
                    
            if(actual[i] == CODE and well_classified == False):
                print(well_classified , "docID:",i,"prediction of ",CODE," was:",predicted[i])
                fail_list_for_code.append(i)
            class_score[actual[i]][1] += 1.0
            length+=1

    avg_acc = 0.0 
    for cl in class_score.keys():
        avg = class_score[cl][0]/class_score[cl][1]
        if(avg<0.5 and count[cl]-class_score[cl][1] > class_score[cl][1]):
            print("!Low precision :! #Correct:", class_score[cl][0], "#Tested:", class_score[cl][1],"#Train",count[cl]-class_score[cl][1])
            fail_list.append(cl)
        print ("\t", cl, "Acc.:", avg, "Correct:", class_score[cl][0], "Tested:", class_score[cl][1],"#Train",count[cl]-class_score[cl][1])
        cumulative_class_score[cl][0] += class_score[cl][0]
        cumulative_class_score[cl][1] += class_score[cl][1]
        
        avg_acc +=avg

    print ('Total Test Examples', length, "\nMicro Acc.(item level)", micro_correct/length)
    global cumulative_micro_avg
    cumulative_micro_avg += micro_correct/length
    return avg_acc/len(class_score)

###############################################################################
# Benchmark classifiers
def benchmark(clf):
    print('_' * 80)
    print("Training: ")
    print(clf)
    t0 = time()
    clf.fit(X_train, y_train)
    train_time = time() - t0
    print("train time: %0.3fs" % train_time)

    t0 = time()

    pred = clf.predict(X_test)	  
    probs = clf.predict_proba(X_test)

    #Right now, it only uses top-5
    for topk in range(5, 6):
        best_n_label = transform_label(clf, probs, topk)

        test_time = time() - t0
        print("test time:  %0.3fs" % test_time)

        pred = best_n_label
        print ("Top-", topk)
        print ("Macro Acc.(class level)", apk_per_class(y_test, best_n_label, topk), "\n\n")


In [13]:

##################################################################
### Define train/test/code list files 
#################################################################
code_list_fn = "NSCC_sample_data_170309_Codelist.dat"
train_fn = "rev_utf8_train_big.dat"
test_fn = "test_5500_from_izip_all_kkma_1col.dat"
all_fn ='izip_kkma_all_3cols.dat'
ENCODING = 'utf-8'

###############################################################################
# Load some categories from the training set
categories =  [x for x in open(code_list_fn,'r').read().split('\n') if len(x) > 0]

print("Loading NSCC	 dataset for categories:")
print(categories if categories else "all")
print(len(categories))

data_train = open(train_fn).readlines()
data_test = open(test_fn).readlines()
all_data = open(all_fn).readlines()

data_train_data, data_test_data = [], []
y_train, y_test = [], []
all_x, all_y = [], []
count = {}

for cat in categories:
    count[cat] = 0

for line in all_data:
    #Shape of training file:CODE + '\t' + name + ' %% ' + kor_kywd + ' %% ' + goal + ' %% ' + abstract + ' %% ' + efct
    items = line.split('\t')
    if len(items) == 2:
        #if document code is not E, ignore that
        if(items[0][0] != 'E'): 
            continue
        all_x.append(items[1].decode(ENCODING, 'ignore'))
        all_y.append(items[0])
        if not count.has_key(items[0]):
            count[items[0]] = 0
        count[items[0]] += 1
    else:
        print('ERROR IN READING')
    """
for line in data_train:
	items = line.split('\t')
	if len(items) == 2:
		data_train_data.append(items[1].decode(ENCODING, 'ignore'))
		y_train.append(items[0])
        if not count.has_key(items[0]):
            count[items[0]] = 0
        count[items[0]] += 1
for line in data_test:
	items = line.split('\t')
	if len(items) == 2:
		data_test_data.append(items[1].decode(ENCODING, 'ignore'))
		y_test.append(items[0])
        if not count.has_key(items[0]):
            count[items[0]] = 0
        count[items[0]] += 1
    """

#divides data to train&test with 9:1 ratio
def divide_data(r_s):
    global data_train_data,data_test_data,y_train,y_test
    data_train_data,data_test_data,y_train,y_test = train_test_split(all_x,all_y,random_state =r_s, train_size = 0.9)
    #trash_a,data_test_data,trash_b,y_test = train_test_split(all_x,all_y,random_state =1, train_size = 0.60)
    print (len(data_train_data), len(data_test_data))
    print('data loaded')
   

Loading NSCC	 dataset for categories:
['EA99', 'EI02', 'EI03', 'EI01', 'EI06', 'EI07', 'EI04', 'EI05', 'EG08', 'EI08', 'EI09', 'EG09', 'EA08', 'EA09', 'EA02', 'EA03', 'EA01', 'EA06', 'EA07', 'EA04', 'EA05', 'EC10', 'EE99', 'EB08', 'EF99', 'EI11', 'EI10', 'EI12', 'EA15', 'EA14', 'EA11', 'EA10', 'EA13', 'EB99', 'EA12', 'EE08', 'EE09', 'EE06', 'EE07', 'EE04', 'EE05', 'EE02', 'EE03', 'EE01', 'EB01', 'EB03', 'EB02', 'EB05', 'EB04', 'EB07', 'EB06', 'EE11', 'EE10', 'EE13', 'EE12', 'EI99', 'EE14', 'EC08', 'EC09', 'EC01', 'EC02', 'EC03', 'EC04', 'EC05', 'EC06', 'EC07', 'EF05', 'EF04', 'EF06', 'EF01', 'EF03', 'EF02', 'ED10', 'ED11', 'EG04', 'EG05', 'EG06', 'EG07', 'EG01', 'EG02', 'EG03', 'ED07', 'ED06', 'ED05', 'ED04', 'ED03', 'ED02', 'ED01', 'ED09', 'ED08', 'EH10', 'EH11', 'EH12', 'EH13', 'EH14', 'EH15', 'EG99', 'EC99', 'ED99', 'EC11', 'EH09', 'EH08', 'EH03', 'EH02', 'EH01', 'EH07', 'EH06', 'EH05', 'EH04', 'EH99', 'EG10']
111


In [14]:
vectorizer = 0
X_train,X_test = 0,0
#vectorize training data using tfidf vectorizer
def vectorize():
    print("Extracting features from the training data using a sparse vectorizer")
    t0 = time()
    my_stop_words = [unicode(x.strip(), 'utf-8') for x in open('kor_stop_word.txt','r').read().split('\n')]

    #print (len(data_train_data))
    global vectorizer,X_train,X_test
    #vectorizer = StemmedTfidfVectorizer(max_df=0.5,stop_words=my_stop_words,max_features = 200000,min_df=5)
    vectorizer = TfidfVectorizer(max_df=0.5,stop_words=my_stop_words,max_features = 200000,min_df=5,ngram_range=(1,1))
    X_train = vectorizer.fit_transform(data_train_data)

    print("n_samples: %d, n_features: %d" % X_train.shape)
    
    print("Extracting features from the training data using a sparse vectorizer")

    X_test = vectorizer.transform(data_test_data)
    print("n_samples: %d, n_features: %d" % X_test.shape)


In [15]:
accumulated_fail_list = {}
TRIAL_SIZE = 3
RANDOM_STATE_START = 5
cumulative_micro_avg = 0
cumulative_class_score = {}
CODE = 'ED99' #code you want to investigate, doesn't affect the result
#try the code with different train/test splts
for t,trial in enumerate(range(RANDOM_STATE_START,RANDOM_STATE_START + TRIAL_SIZE)):
    print("Trial:",t,"R_s",trial)
    divide_data(trial)
    print("Vectorizing: ",trial)
    vectorize()
    feature_names = vectorizer.get_feature_names()
    fail_list = []
    fail_list_for_code = []
    feature_names = np.asarray(feature_names)
    # Train SGD model
    print("Learning: ",trial)
    
    #suggested by sklearn tutorial:http://scikit-learn.org/stable/modules/sgd.html#tips-on-practical-use
    suggested_n_iter = np.ceil(10**6/len(data_train_data)) 
    
    #DONT USE ELASTICNET as penalty, it will give lower accuracy
    clf = SGDClassifier(loss='modified_huber', alpha=0.0001, n_iter=10, penalty='l2')
    benchmark(clf)
    for x in fail_list:
        if not accumulated_fail_list.has_key(x):
            accumulated_fail_list[x] = 0
        accumulated_fail_list[x] +=1
    #print(fail_list)
    for x in accumulated_fail_list:
        if accumulated_fail_list[x] == TRIAL_SIZE:
            print(x,accumulated_fail_list[x])
#print the average micro accuracy            
print('Cumulative',cumulative_micro_avg/TRIAL_SIZE)

Trial: 0 R_s 5
54297 6033
data loaded
Vectorizing:  5
Extracting features from the training data using a sparse vectorizer
n_samples: 54297, n_features: 89815
Extracting features from the training data using a sparse vectorizer
n_samples: 6033, n_features: 89815
Learning:  5
________________________________________________________________________________
Training: 
SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='modified_huber', n_iter=10, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=False)
train time: 29.207s
test time:  0.416s
Top- 5
False docID: 551 prediction of  ED99  was: ['EF99' 'EA07' 'EA02' 'EA09' 'ED05']
False docID: 1415 prediction of  ED99  was: ['ED04' 'EB06' 'ED05' 'EB01' 'ED10']
False docID: 1668 prediction of  ED99  was: ['EI03' 'EE99' 'EE07' 'EE11' 'EE02']
False docID: 1883 prediction of  ED99 

n_samples: 6033, n_features: 89974
Learning:  6
________________________________________________________________________________
Training: 
SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='modified_huber', n_iter=10, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=False)
train time: 30.936s
test time:  0.431s
Top- 5
False docID: 245 prediction of  ED99  was: ['EA07' 'EA09' 'ED01' 'EA10' 'ED06']
False docID: 487 prediction of  ED99  was: ['EI11' 'EE09' 'EA14' 'EE02' 'EE13']
False docID: 959 prediction of  ED99  was: ['EF04' 'EI09' 'EF03' 'EI03' 'EF06']
False docID: 1188 prediction of  ED99  was: ['EE05' 'ED11' 'EE12' 'ED04' 'EA09']
False docID: 1372 prediction of  ED99  was: ['EE04' 'EE02' 'EE01' 'EE06' 'ED11']
False docID: 1375 prediction of  ED99  was: ['EA09' 'EI05' 'EE01' 'EE99' 'ED08']
False docID: 1445 predict

n_samples: 6033, n_features: 90093
Learning:  7
________________________________________________________________________________
Training: 
SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='modified_huber', n_iter=10, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=False)
train time: 28.615s
test time:  0.375s
Top- 5
False docID: 202 prediction of  ED99  was: ['ED06' 'EA08' 'EA07' 'EG07' 'ED01']
False docID: 291 prediction of  ED99  was: ['EB03' 'EC09' 'EA03' 'ED09' 'EA09']
False docID: 462 prediction of  ED99  was: ['EA05' 'EA08' 'EE03' 'ED07' 'ED08']
False docID: 624 prediction of  ED99  was: ['EA99' 'EA02' 'ED10' 'EE99' 'EE02']
False docID: 633 prediction of  ED99  was: ['EB03' 'ED02' 'EC05' 'ED01' 'EF06']
False docID: 1950 prediction of  ED99  was: ['EA10' 'EE05' 'EE06' 'EE99' 'EE09']
False docID: 2046 predictio

In [11]:
#this block prints the statistics(info about each category)
statistics = open('statistics_3rdtest.txt','w')
statistics.write('\t'.join(['CODE','ALL','TRAIN','TEST','CORRECT','correct%'])+'\n')

for x in count:
    a_row = [x,count[x],count[x]-cumulative_class_score[x][1],cumulative_class_score[x][1],cumulative_class_score[x][0],cumulative_class_score[x][0]/cumulative_class_score[x][1]]#code,train,tested,correct
    for i in range(len(a_row)):
        a_row[i] = str(a_row[i])
    statistics.write('\t'.join(a_row)+'\n')
statistics.close()

In [21]:
#which categories failed most
for x in accumulated_fail_list:
        if accumulated_fail_list[x] >= TRIAL_SIZE -2:
            print(x,accumulated_fail_list[x])

EH12 1
EA99 1
EH15 1
ED99 1


In [22]:
#which categories failed most 2
for x in cumulative_class_score:
    if(x[0] == "E"):
        if(cumulative_class_score[x][0]/cumulative_class_score[x][1]<0.5):
            print(x,cumulative_class_score[x],count[x]*TRIAL_SIZE)

EA99 [20.0, 41.0] 372
EH12 [4.0, 10.0] 106
EH15 [2.0, 5.0] 25
ED99 [14.0, 40.0] 582


In [23]:
#get the index for specific code
# EH14 is 95
# EH15 is 96
# EA99 is 15
# ed99 is 48
# ee00 is 63
print(suggested_n_iter)
CODE_INDEX = clf.classes_.tolist().index(CODE)
CODE_INDEX

20.0


48

In [24]:
#which words have high weight for CODE_INDEX
white_list = []
for i,x in enumerate(vectorizer.get_feature_names()):
    if(clf.coef_[CODE_INDEX][i]>0.5):
        print(clf.coef_[CODE_INDEX][i],x)
        white_list.append(x)

0.517542185785 endoscopi
0.554998145394 lamp
0.551419272672 triangular
0.563239502759 turnov
0.608646924589 wearabl
0.66730913174 공급기
0.519867299344 광시야
0.925674999351 리졸버
0.602099061057 산업원천기술개발
0.746020559369 수송시스템산업핵심
0.79116022665 시설보수
0.621038667741 시스템반도체
0.520720374026 실무
0.520453219473 연주
0.596783766286 유연인쇄전자
0.633231151025 융복합시스템
0.608676919415 인포
0.644928267792 전기설비
0.656657027617 전자정보디바이스
0.533554481677 전자파
0.559106759631 주름패턴
0.500680965876 지벡계수
0.562054476646 지역산업
0.749688808711 창의인재양성
0.586173454145 채용
0.580687250659 충청권
0.563884064454 포트
0.514010717075 품목
0.618096097616 학연산


In [14]:
out = file("features.txt","w")

for x in feature_names:
    out.write(x.encode('utf-8','ignore')+"\n")
out.close()

In [18]:
print(y_test[fail_list_for_code[0]],data_test_data[fail_list_for_code[0]])
fail_data = []
for x in fail_list_for_code:
    fail_data.append(data_test_data[x])

ED99 태양 광전지판 이용 탄소 섬유 온 열매트 전용 난방 시스템 기술 개발 태양광 발전 전기 탄소 섬유 전기 매트 재생 에너지  sunlight generation electricity carbon fiber electricity mat new regeneration energy 태양 전지판 이용 가정 난방 탄소 섬유 전 기온 열매트 전용 난방 시스템 개발 태양 광전지판 인버터 사용 태양 광전지판 발생 전기 축전 전류 사용 난방 시스템 개발 전기 온 열매트 난방 온도 최고 이상 발생 안전 지속 난방 가능 시스템 개발 태양 전용 탄소 섬유 전 기온 열매트 설계 개발 축전지 전력 저장 한도 난방 설계 매트 최고 난방 온도 이상 매트 난방 유지 온도 태양 전지판 전력 발생량 이상 최대 출력 동작 전압 최대 출력 동작 전류 축전지 지속 시간 탄소 섬유 매트 전자파 이하 태양광 집열 전지판 용도 규격 개발 집광판 이용 이상 용량 태양열 전지판 직열 어레이 구성 열매트 인 최고 온도 이상 유지 용도 크기 규격 설계 개발 아파트 베란다 설치 집광판 규격 개발 규격 적정 전기 용량 데이터 확보 아파트 베란다 길이 축전지 시스템 개발 축전 전기 저장 장치 설치 인버트 장치 설치 방법 개발 전기 전환 장치 개발 태양광 전기 일반 전기 교체 사용 전기 전환 장치 개발 전기 난방 배선 시스템 개발 비상등 배선 방법 연결 코드 설치 방법 개발 아파트 구조 일반 주택 구조 실내 전기 배선 설치 설계 방법 태양 광 전기 전용 탄소 섬유 온 열매트 개발 전기 온열매트 개발 사용 가능 매트 개발 원적외선 발생 은나노 섬유 황토 코팅 원단 개발 온도 조절기 개발 겸용 태양 광 전기 온열매트 온도 조절기 개발 온도 조절 장치 조절기 제품 디자인 개발 시 작품 제작 기술 개발 효과 아파트 주택 전기 사용량 누진 적용 전력 사용 가정 전기 요금 부과 태양 광전기 일부 사용 전기 요금 절감 효과 예 월 전기 사용량 설비 기준 주택 경우 월 전기 요금 부과 태양광 설치 부담 절감 월 전기 사용량 

In [19]:
v = vectorizer.transform(fail_data)
v_array = v.toarray()
important_word = []
for i in range(200000):
    important_word.append([])
buffer = 200000*[0]
print("Fail size:",len(fail_data))
for i,doc in enumerate(fail_data):
    print(doc)
    #print(v.toarray()[i])
    #rev_list =  reversed(np.argsort(v.toarray()[i]))
    for j in range(len(v_array[i])):
        if(v_array[i][j]>0.0):
     #       print(index,feature_names[index])
            buffer[j] = 1
        
    for j in range(len(buffer)):
        if buffer[j] > 0:
            important_word[j].append(i)
            buffer[j] = 0
            


Fail size: 29
태양 광전지판 이용 탄소 섬유 온 열매트 전용 난방 시스템 기술 개발 태양광 발전 전기 탄소 섬유 전기 매트 재생 에너지  sunlight generation electricity carbon fiber electricity mat new regeneration energy 태양 전지판 이용 가정 난방 탄소 섬유 전 기온 열매트 전용 난방 시스템 개발 태양 광전지판 인버터 사용 태양 광전지판 발생 전기 축전 전류 사용 난방 시스템 개발 전기 온 열매트 난방 온도 최고 이상 발생 안전 지속 난방 가능 시스템 개발 태양 전용 탄소 섬유 전 기온 열매트 설계 개발 축전지 전력 저장 한도 난방 설계 매트 최고 난방 온도 이상 매트 난방 유지 온도 태양 전지판 전력 발생량 이상 최대 출력 동작 전압 최대 출력 동작 전류 축전지 지속 시간 탄소 섬유 매트 전자파 이하 태양광 집열 전지판 용도 규격 개발 집광판 이용 이상 용량 태양열 전지판 직열 어레이 구성 열매트 인 최고 온도 이상 유지 용도 크기 규격 설계 개발 아파트 베란다 설치 집광판 규격 개발 규격 적정 전기 용량 데이터 확보 아파트 베란다 길이 축전지 시스템 개발 축전 전기 저장 장치 설치 인버트 장치 설치 방법 개발 전기 전환 장치 개발 태양광 전기 일반 전기 교체 사용 전기 전환 장치 개발 전기 난방 배선 시스템 개발 비상등 배선 방법 연결 코드 설치 방법 개발 아파트 구조 일반 주택 구조 실내 전기 배선 설치 설계 방법 태양 광 전기 전용 탄소 섬유 온 열매트 개발 전기 온열매트 개발 사용 가능 매트 개발 원적외선 발생 은나노 섬유 황토 코팅 원단 개발 온도 조절기 개발 겸용 태양 광 전기 온열매트 온도 조절기 개발 온도 조절 장치 조절기 제품 디자인 개발 시 작품 제작 기술 개발 효과 아파트 주택 전기 사용량 누진 적용 전력 사용 가정 전기 요금 부과 태양 광전기 일부 사용 전기 요금 절감 효과 예 월 전기 사용량 설비 기준 주택 경우 월 전기 요금 부과 태양광 설치 부담 절감 

열화 카메라 이용 지능 화재 감시 시스템 개발 광범위 화재 감시 신뢰 화재 상황 모니터링 비용 열화 카메라 지능 화재 감시  wide fire surveillance high reliability fire monitoring low cost thermal camera intelligent fire detection 기존 화재 감지 시스템 연기 감지기 감지기 불꽃 감지기 한계 점 극복 문제점 해결 열화 카메라 이용 정확 신속 대응 가능 지능 화재 감시 시스템 개발 원거리 발생 화재 초기 감지 기술 개발 사각 지역 회전 방식 화재 감시 기반 파노라마 화재 감시 기술 개발 알람 화재 감지 정확 향상 기술 개발 중저 화질 영상 고가 화질 영상 카메라 유사 성능 발휘 중저가 임베디드 시스템 구축 카메라 이하 기존 화재 감지기 대체 지역 상대 개수 열화상 센서 모니터링 가능 화재 가능 공장 공공시설 적용 신속 정확 화재 대응 예방 원거리 발생 화재 감지 화재 초기 진압 가능 화재 감지 시스템 수출 지 재산 기술료 징수 이익 창출 

소형 선박 위치 향상 안전 정보 문자 서비스 장치 개발 위성 항법 지피에스 컨수신기 소형 선박 보정 정보  d gps beacon g nss 시스템 통합 보정 정보 연산 처리부 소형 설계 개발 신호 처리 알고리즘 개발 통합 수신기 상세 설계 검증 시제 개발 해양 안전 경보기 상세 설계 검증 시제 개발 통합 수신기 디자인 목업 개발 해양 안전 경보기 디자인 목업 개발 시스템 통합 보정 정보 연산 처리부 상용모듈 개발 안테나부 부 수신부 모듈 개발 중파 획득 위성 항법 보정 신호 처리 알고리즘 안정 통합 수신기 시제품 개발 해양 안전 경보기 시제품 개발 통합 수신기 금형 기구 개발 해양 안전 경보기 금형 기구 개발 앱 연동 소프트웨어 개발 종합 시험 평가 장비 인증 개발 시스템 통합 보정 정보 연산 처리부 소형 시제품 설계 규격 통합 단말기 시스템 구성 소형 설계 중파 보정 정보 신호 처리 알고리즘 개발 보정 정보 서비스 처리부 프로토콜 개발 통

상용 소형 광학 레이더 기반 충돌 회피 시스템 개발 레이다 무인기 연속 주파 충돌 회피 영상 처리  radar u av fm cw collision avoid image detect 과제 최종 목표 사용 가능 소형 주파수 모듈레이션 연속파 방식 레이더 활용 충돌 방지 정보 제공 광학식 카메라 모듈 장착 추가 정보 제공 과제 개발 하드웨어 다음 밴드 안테나 방식 주파 레이더 송수신기 증폭기 회로 고속 회로 고속 회로 신호 처리 처리부 레이더 제어 데이터 처리 연산 레이더 검출 연산 외부 인터페이스 제공 충돌 회피 관련 소프트웨어 카메라 영상 정보 추출 레이더 센서 기반 전처리 장애물 탐지 모듈 소프트웨어 개발 레이더 센서 기반 파형 설계 발생 장애물 탐지 검증 소프트웨어 개발 광학 센서 기반 장애물 탐지 모듈 과제 사용 카메라 시스템 라즈베리파이 카메라 시스템 카메라 장착 일체 카메라 무게 급 해상도 제공 레이더 센서 기반 장애물 탐지 모듈 개발 설치 레이더 하드웨어 무게 소비 전력 고려 일반 사용 방식 펄스파 도플러 레이더 방식 주파수 모듈레이션 연속파 레이더 방식 안테나본 과제 소출력 전파 사용 마이크로스트립 패치 안테나 적용 예정 송출부 수신부 레이더 다음 시간 선 스윕 삼각파 변조 송신 장애물 반사 수신 신호 송신 신호 곱 주파수 차주 파수 차주 파수 거리 수신 신호 처리부 레이더 수신 처리부 수신부 수신 신호 최종 해석 모듈 수신부 송출 신호 수신 신호 모듈레이션 신호 최종 출력 신호 고속 디지털 신호 주파수 성분 추출 주파수 성분 추출 기법 사용 고속 처리 신호 처리부 인터페이스 과제 개발 목표 모듈 방식 레이더 일반 시스템 구매 조립 사용 범용 통신 인터페이스 제공 검토 방식 일반 시리얼 인터페이스 버스 방식 범용 인터페이스 제공 예정 전처리 모듈 입력 신호 디지털 데이터 저장 요구 시작 끝 시점 지정 제어 신호 변환 상호 동기 과정 데이터 저장 장애물 탐지 모듈 전 처리 모듈 전달 데이터 장애물 거리 산출 모듈 수신 값 이용 측정 송신 시점 수신 

In [22]:
for i,count in enumerate(important_word):
    if len(count) > 10:
        print(len(count),count,feature_names[i],clf.coef_[63][i])

17 [1, 2, 4, 8, 9, 10, 11, 13, 18, 19, 20, 21, 22, 23, 25, 26, 28] 관련 -0.0589689547829
12 [0, 3, 5, 6, 7, 10, 14, 17, 20, 21, 24, 27] 구조 -0.014681700471
11 [1, 4, 11, 12, 15, 19, 20, 21, 23, 24, 26] 구축 0.00160561546174
12 [6, 7, 8, 10, 11, 12, 13, 16, 18, 26, 27, 28] 성능 -0.0459541954862
11 [4, 7, 8, 10, 12, 15, 17, 22, 24, 26, 27] 센서 -0.0929137712641
16 [1, 2, 4, 8, 9, 10, 11, 13, 14, 16, 18, 19, 22, 24, 25, 26] 시장 0.204275988193
15 [0, 5, 6, 7, 9, 10, 11, 16, 17, 18, 22, 24, 25, 27, 28] 제작 -0.106692923436
14 [0, 2, 4, 5, 6, 9, 11, 14, 15, 16, 18, 22, 24, 26] 제품 -0.0150238899123
16 [0, 1, 2, 3, 7, 8, 9, 13, 14, 16, 18, 20, 24, 25, 26, 27] 효과 0.0356542569683
